In [3]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

from config import api_key

import numpy as np
import requests
import json

In [133]:
# Import files as DataFrames
fourteen_fiftenn_pd = pd.read_csv("draft_data/2014-15.csv")
seventeen_eighteen_pd = pd.read_csv("draft_data/2017-18.csv", encoding = "ISO-8859-1")
twenty_twentyone = pd.read_csv("draft_data/2020-21.csv", encoding = "ISO-8859-1")

In [134]:
# Rename columns to be uniform across dataframes
twenty_twentyone.rename(columns={"Facility name":"Facility Name"}, inplace=True)

In [135]:
# Remove rows of corporate total data
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Facility Name'] != 'Corporate Total']
seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Facility Name'] != 'Corporate Total']
twenty_twentyone = twenty_twentyone[twenty_twentyone['Facility Name'] != 'Corporate Total']

In [136]:
# Drop columns of un-needed data
fourteen_fiftenn_pd = fourteen_fiftenn_pd.drop(columns={"Type", "Scope 1 (t CO2-e)","Scope 2 (t CO2-e)","Total Emissions (t CO2-e)", "Emission  Intensity (t/Mwh)", "Grid Connected", "Grid", "Important Notes"})
seventeen_eighteen_pd = seventeen_eighteen_pd.drop(columns={"Type", "Total Scope 1 Emissions \n(t CO2-e)","Total Scope 2 Emissions \n(t CO2-e)","Total Emissions \n(t CO2-e)", "Emission Intensity \n(t CO2-e/ MWh)", "Grid Connected", "Grid"})
twenty_twentyone = twenty_twentyone.drop(columns={"Type", "Total scope 1 emissions \n(t CO2-e)","Total scope 2 emissions \n(t CO2-e)","Total emissions \n(t CO2-e)", "Emission intensity \n(t CO2-e/ MWh)", "Grid connected", "Grid"})


In [137]:
# Add year column for filtering in plots
fourteen_fiftenn_pd['Year'] = '2014-2015'

seventeen_eighteen_pd['Year'] = '2017-2018'

twenty_twentyone['Year'] = '2020-2021'

In [138]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fourteen_fiftenn_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fourteen_fiftenn_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fourteen_fiftenn_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fourteen_fiftenn_pd.loc[index, "Lat"] = "NaN"
        fourteen_fiftenn_pd.loc[index, "Lng"] = "NaN"

# Visualize to confirm lat lng appear
fourteen_fiftenn_pd.head()

,Reporting Entity,Facility Name,State,Electricity Production (GJ),Electricity Production (Mwh),Primary Fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,567719.0,157700.0,Wind,2014-2015,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,ACT,213115.0,59199.0,Solar,2014-2015,-35.4898,149.143
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,2461803.0,683834.0,Wind,2014-2015,-37.3946,143.639
4,AGL ENERGY LIMITED,Banimboola Hydro,VIC,137094.0,38082.0,Hydro,2014-2015,-36.5343,147.459
5,AGL ENERGY LIMITED,Bayswater Power Station,NSW,50275548.0,13965430.0,Black Coal,2014-2015,-32.3926,150.949


In [139]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in seventeen_eighteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        seventeen_eighteen_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        seventeen_eighteen_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        seventeen_eighteen_pd.loc[index, "Lat"] = "NaN"
        seventeen_eighteen_pd.loc[index, "Lng"] = "NaN"

seventeen_eighteen_pd.head()

,Reporting Entity,Facility Name,State,Electricity Production (GJ),Electricity Production (MWh),Primary Fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Cathedral Rocks Wind Farm,SA,"550,950","153,042",Wind,2017-2018,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,NSW,"601,740","167,150",Wind,2017-2018,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Mt Gellibrand Wind Farm,VIC,"4,320","1,200",Wind,2017-2018,-38.2328,143.792
3,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,VIC,"2,317,673","643,798",Wind,2017-2018,-37.3946,143.639
5,AGL ENERGY LIMITED,Banimboola Hydro,VIC,"11,089","3,080",Hydro,2017-2018,-36.5343,147.459


In [140]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twenty_twentyone.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location} + {state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twenty_twentyone.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twenty_twentyone.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twenty_twentyone.loc[index, "Lat"] = "NaN"
        twenty_twentyone.loc[index, "Lng"] = "NaN"

twenty_twentyone.head()

,Reporting entity,Facility Name,State,Electricity production (GJ),Electricity production (MWh),Primary fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,SA,"512,455","142,348",Wind,2020-2021,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,"575,522","159,867",Wind,2020-2021,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,VIC,"1,018,498","282,916",Wind,2020-2021,-38.2328,143.792
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,"2,339,624","649,896",Wind,2020-2021,-37.3946,143.639
5,ADANI RENEWABLE ASSET HOLDINGS PTY LTD,Adani Rugby Run Solar Farm,QLD,"587,856","163,293",Solar,2020-2021,-22.1441,147.914


In [141]:
# Drop rows which returned no coordinates for facilities
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Lat'] == 'NaN']

seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Lat'] == 'NaN']

twenty_twentyone = twenty_twentyone[twenty_twentyone['Lat'] == 'NaN']

In [ ]:
# fourteen_fifteen, seventeen_eighteen, twenty_twentyone
# filter each df to group by state and then group by again to show primary fuel types

In [ ]:
# // Energy Tables: Titled 14_15_Energy_Production, 17_18_Energy_Production & 20_21_Energy_Production
# // Energy Tables Column Headers:
# // # Reporting entity
# // # Facility name
# // # State
# // # Electricity production (GJ)
# // # Electricity production (MWh)

# DELETE THESE COLUMNS
# // # Total scope 1 emissions (t CO2-e)
# // # Total scope 2 emissions (t CO2-e)
# // # Total emissions (t CO2-e)
# // # Emission intensity (t CO2-e/ MWh)
# // # Grid connected
# // # Grid

# // # Primary fuel
# Lat
# Long
# // # Year

# // Population Table Titled Aus_Population
# // Aus_Population Column Headers:
# // # State
# // # Male
# // # Female
# // # Total

# // Income Table Titled Aus_Income
# // Aus_Income Column Headers:
# // # State
# // # Median Income

In [ ]:
fourteen_fiftenn_pd.head(2)

In [ ]:
fourteen_fiftenn_pd

In [142]:
# fourteen_fifteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# fourteen_fifteen_transformed = fourteen_fiftenn_pd[fourteen_fifteen_cols].copy()

fourteen_fifteen_transformed = fourteen_fiftenn_pd.rename(columns={"Reporting Entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (Mwh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [143]:
# seventeen_eighteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# seventeen_eighteen_transformed = seventeen_eighteen_pd[seventeen_eighteen_cols].copy()

seventeen_eighteen_transformed = fourteen_fiftenn_pd.rename(columns={"Reporting Entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (Mwh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [144]:
# seventeen_eighteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# seventeen_eighteen_transformed = seventeen_eighteen_pd[seventeen_eighteen_cols].copy()

seventeen_eighteen_transformed = fourteen_fiftenn_pd.rename(columns={"Reporting Entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (Mwh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [4]:
connection_string = "postgres:postgres@localhost:5432/energy_db"
engine = create_engine(f'postgresql://{connection_string}')

In [1]:
import psycopg2

In [8]:
# Confirm tables
engine.table_names()

/var/folders/2z/kfk1f34j2tl95byv9wl1css80000gn/T/ipykernel_78259/3015872754.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
fourteen_fifteen_transformed.to_sql(name='fourteen_fifteen', con=engine, if_exists='append', index=True)

In [ ]:
seventeen_eighteen_transformed.to_sql(name='seventeen_eighteen', con=engine, if_exists='append', index=True)

In [ ]:
twenty_twentyone.to_sql(name='twenty_twentyone', con=engine, if_exists='append', index=True)